## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-06-16-40-13 +0000,nyt,China’s National Security Office in Hong Kong ...,https://www.nytimes.com/2025/12/06/world/asia/...
1,2025-12-06-16-34-27 +0000,nyt,Have Trump’s Tariffs Gone as High as They Can ...,https://www.nytimes.com/2025/12/06/business/ec...
2,2025-12-06-16-33-22 +0000,nypost,Zohran Mamdani’s DSA party pushing China propa...,https://nypost.com/2025/12/06/us-news/dsa-push...
3,2025-12-06-16-30-24 +0000,nypost,Trump rips ‘nasty’ Kaitlan Collins and touts ‘...,https://nypost.com/2025/12/06/us-news/trump-ri...
4,2025-12-06-16-28-52 +0000,nypost,Woman allegedly left by boyfriend to freeze to...,https://nypost.com/2025/12/06/world-news/woman...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2341/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,42
250,netflix,16
252,warner,15
23,new,13
429,court,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
222,2025-12-05-19-14-25 +0000,latimes,Supreme Court will hear Trump's plan to restri...,https://www.latimes.com/politics/story/2025-12...,93
207,2025-12-05-19-51-16 +0000,cbc,U.S. Supreme Court to decide whether Trump's b...,https://www.cbc.ca/news/world/supreme-court-tr...,85
213,2025-12-05-19-33-39 +0000,nypost,Supreme Court to decide whether or not Trump’s...,https://nypost.com/2025/12/05/us-news/supreme-...,84
164,2025-12-05-22-05-51 +0000,nyt,Supreme Court Agrees to Review Trump Order Res...,https://www.nytimes.com/2025/12/05/us/politics...,83
196,2025-12-05-20-34-00 +0000,wsj,Supreme Court to Decide if Trump Birthright Ci...,https://www.wsj.com/us-news/law/supreme-court-...,82


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
222,93,2025-12-05-19-14-25 +0000,latimes,Supreme Court will hear Trump's plan to restri...,https://www.latimes.com/politics/story/2025-12...
168,60,2025-12-05-21-56-47 +0000,nyt,Netflix to Buy Warner Bros in $83 Billion Deal,https://www.nytimes.com/2025/12/05/business/wa...
165,41,2025-12-05-21-59-27 +0000,wapo,Olivia Nuzzi will leave Vanity Fair amid new a...,https://www.washingtonpost.com/business/2025/1...
90,39,2025-12-06-04-27-00 +0000,wsj,A key vaccine panel at the Centers for Disease...,https://www.wsj.com/politics/policy/cdc-panel-...
112,38,2025-12-06-01-02-00 +0000,wsj,Two survivors of a Sept. 2 lethal U.S. strike ...,https://www.wsj.com/politics/national-security...
44,37,2025-12-06-12-52-57 +0000,nypost,National Guard shooting suspect’s close milita...,https://nypost.com/2025/12/06/us-news/national...
29,33,2025-12-06-14-35-09 +0000,nypost,Stephen Miller calls billion-dollar Minnesota ...,https://nypost.com/2025/12/06/us-news/stephen-...
193,32,2025-12-05-20-48-41 +0000,nypost,Ex-NYC Council staffers threatened with legal ...,https://nypost.com/2025/12/05/us-news/favorite...
53,31,2025-12-06-11-45-00 +0000,nypost,Russia unleashes massive drone and missile att...,https://nypost.com/2025/12/06/world-news/russi...
88,31,2025-12-06-05-14-15 +0000,nypost,White House slams Newsom after gov claims Trum...,https://nypost.com/2025/12/06/us-news/white-ho...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
